# GCP Analysis

* Module Name: gcp.ipynb
* Description: ground control points are used to calibrate the performance of multi camera homography

Copyright (C) 2025 J.Cincotta

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program. If not, see <https://www.gnu.org/licenses/>.


In [1]:
"""
Configuration for this Notebook:
"""
filenames: list[str] = [
    "camera-gcp-tool/data/c1-export.csv",
    "camera-gcp-tool/data/c2-export.csv",
    "camera-gcp-tool/data/c3-export.csv"
]


## Import data 

CSV loader.


In [10]:
import csv


def load_csv(filename):
    output: dict = {}
    row_count = 0
    with open(filename, "r") as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            output[row["gcp"]] = (int(row["transformed-x"]), int(row["transformed-z"]))
            row_count += 1
        print(f"Imported {filename} with {row_count} rows.")
    return output

cameras: list = []
for filename in filenames:
    cameras.append(load_csv(filename))


Imported camera-gcp-tool/data/c1-export.csv with 7 rows.
Imported camera-gcp-tool/data/c2-export.csv with 7 rows.
Imported camera-gcp-tool/data/c3-export.csv with 7 rows.
[{'0': (-1, -1), '1': (10, -1), '2': (-1, 11), '3': (11, 10), '4': (50, -28), '5': (-17, -8), '6': (62, 78)}, {'0': (-1, -1), '1': (11, -2), '2': (-1, 10), '3': (10, 10), '4': (54, -39), '5': (-21, -24), '6': (68, 71)}, {'0': (-1, -1), '1': (11, -1), '2': (-2, 11), '3': (10, 10), '4': (54, -42), '5': (-20, -23), '6': (77, 84)}]


## Mapping of GCP to Homography

We know the measurements of the calibration square are 1800 x 1800. In our homographic transform, we also know this equates to 10x10 units in the homograpic lookup (homographic units). Therefore, 180mm is 1 homographic unit in our mapping model.

This makes the first four coordinates of the GCP are:
* GCP-1 = 0,0
* GCP-2 = 10,0
* GCP-3 = 0,10
* GCP-4 = 10,10

We can validate the other GCP using this insight and given we know the euclidean distance from 0,0 to each of the other GCP on the map, we should see the following ratios agreed with (or not!):
* GCP-1 to GCP-4 = 14.14 homographic units (2546mm)
* GCP-1 to GCP-5 = 56.56 homographic units (10181mm)
* GCP-1 to GCP-6 = 32.23 homographic units (5801mm)
* GCP-1 to GCP-7 = 85.28 homographic units (15350mm)


In [17]:
# based on the above knowledge, we set up a known distances
known_distances: list = [0, 10, 10, 14.14, 56.56, 32.23, 85.28]

## Calculate homographic distances


In [18]:
def euclidan_distance(src,dst) -> float:
    src_x, src_z = src
    dst_x, dst_z = dst
    return (((dst_x - src_x) ** 2) + ((dst_z - src_z) ** 2)) ** 0.5

output = []
for camera, gcps in enumerate(cameras):
    output.append({})
    basis = gcps["0"]
    for index, gcp in enumerate(gcps.keys()):
        output[camera][gcp] = euclidan_distance(basis, gcps[gcp])
        

## Data validation (dumps)

In [19]:
for gcps in cameras:
    print(gcps)

for eds in output:
    print(eds)

{'0': (-1, -1), '1': (10, -1), '2': (-1, 11), '3': (11, 10), '4': (50, -28), '5': (-17, -8), '6': (62, 78)}
{'0': (-1, -1), '1': (11, -2), '2': (-1, 10), '3': (10, 10), '4': (54, -39), '5': (-21, -24), '6': (68, 71)}
{'0': (-1, -1), '1': (11, -1), '2': (-2, 11), '3': (10, 10), '4': (54, -42), '5': (-20, -23), '6': (77, 84)}
{'0': 0.0, '1': 11.0, '2': 12.0, '3': 16.278820596099706, '4': 57.706152185014034, '5': 17.46424919657298, '6': 101.04454463255303}
{'0': 0.0, '1': 12.041594578792296, '2': 11.0, '3': 15.556349186104045, '4': 66.85057965343307, '5': 30.479501308256342, '6': 99.72462083156798}
{'0': 0.0, '1': 12.0, '2': 12.041594578792296, '3': 15.556349186104045, '4': 68.60029154456998, '5': 29.068883707497267, '6': 115.36463929644994}


## Output for table


In [30]:
gcp_distances: dict = {}


for camera, gcps in enumerate(output):
    print(f"Results for camera {camera + 1}")
    distances = []
    for index, value in enumerate(gcps.values()):
        d_sub = gcp_distances.get(f"GCP-1 to GCP-{index+1}", [])
        d = known_distances[index] - value
        print(f"GCP-1 to GCP-{index+1} error: {d}")
        distances.append(d)
        d_sub.append(d)
        gcp_distances[f"GCP-1 to GCP-{index+1}"] = d_sub
    print(f"RMSE is {((sum([x**2 for x in distances]) ** 0.5) / len(distances))}")
    print()
    print()

for dkey in gcp_distances.keys():
    print(f"RMSE for {dkey} is {((sum([x**2 for x in gcp_distances[dkey]]) ** 0.5) / len(gcp_distances[dkey]))}")
    

Results for camera 1
GCP-1 to GCP-1 error: 0.0
GCP-1 to GCP-2 error: -1.0
GCP-1 to GCP-3 error: -2.0
GCP-1 to GCP-4 error: -2.1388205960997055
GCP-1 to GCP-5 error: -1.1461521850140315
GCP-1 to GCP-6 error: 14.765750803427018
GCP-1 to GCP-7 error: -15.764544632553026
RMSE is 3.121474280914666


Results for camera 2
GCP-1 to GCP-1 error: 0.0
GCP-1 to GCP-2 error: -2.0415945787922958
GCP-1 to GCP-3 error: -1.0
GCP-1 to GCP-4 error: -1.4163491861040445
GCP-1 to GCP-5 error: -10.29057965343307
GCP-1 to GCP-6 error: 1.7504986917436547
GCP-1 to GCP-7 error: -14.444620831567974
RMSE is 2.574527892851093


Results for camera 3
GCP-1 to GCP-1 error: 0.0
GCP-1 to GCP-2 error: -2.0
GCP-1 to GCP-3 error: -2.0415945787922958
GCP-1 to GCP-4 error: -1.4163491861040445
GCP-1 to GCP-5 error: -12.040291544569982
GCP-1 to GCP-6 error: 3.1611162925027294
GCP-1 to GCP-7 error: -30.084639296449936
RMSE is 4.6734616627174566


RMSE for GCP-1 to GCP-1 is 0.0
RMSE for GCP-1 to GCP-2 is 1.0092961477162683
RMSE 